In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
from tkcalendar import Calendar
from datetime import datetime, timedelta

# Globale Konstanten für die Ticketpreise, Rabatte und Zusatzoptionen
STANDARDPREIS = {
    "2 Stunden": 24,
    "4 Stunden": 32,
    "6 Stunden": 40,
    "Tageskarte": 49
}

RABATT = {
    "Schüler/Student": 0.15,
    "Senior": 0.10,
    "Kind (0-5 Jahre)": 0.20,
    "Behindert": 0.25
}

ZUSATZPREIS = {
    "Wellenbad": 10,
    "Sauna": 15,
    "Keine": 0
}

RABATT_WINTERZEIT = 0.10  # Rabatt während der Winterzeit
WINTERMONATE = {1, 2, 3, 4, 10, 11, 12}  # Monate, die als Winterzeit gelten

# Listen für die Werte der Comboboxen
AUFENTHALT = ["2 Stunden", "4 Stunden", "6 Stunden", "Tageskarte"]
STATUS = ["Erwachsener", "Schüler/Student", "Senior", "Kind (0-5 Jahre)", "Behindert"]
ZUSATZ = ["Keine", "Wellenbad", "Sauna"]

# Globale Variablen für den gesamten Preis und die Tickets
gesamt_preis = [] #Diese Variable ist eine leere Liste ([]), die im Laufe des 
#Programms verwendet wird, um die Preise der einzelnen ausgewählten Tickets 
#zu speichern. Jedes Mal, wenn ein Ticket hinzugefügt wird, wird sein Preis zur
#Liste gesamt_preis hinzugefügt. Am Ende wird die Summe aller Preise in dieser 
#Liste berechnet und für die Zahlungsfunktion übergeben.
tickets = [] #Auch dies ist eine leere Liste ([]), die verwendet wird, 
#um Details über jedes ausgewählte Ticket zu speichern. 
#Jedes Mal, wenn ein Ticket hinzugefügt wird (mit Informationen wie Preis, 
#Aufenthaltsdauer, Personengruppe, Zusatzoptionen, Datum und Startzeit), 
#wird eine Tupel oder Liste mit diesen Informationen zur Liste tickets 
#hinzugefügt. Diese Liste tickets wird später verwendet, um Zahlungs- und 
#Rechnungsinformationen anzuzeigen.

# Hauptfenster erstellen
window = tk.Tk()
window.title("Preiskalkulator")

# Funktion zum Zentrieren des Fensters auf dem Bildschirm
def fenster_einstellen(fenster, breite, höhe):
    bildschirm_breite = fenster.winfo_screenwidth()  # Bildschirmbreite abrufen
    bildschirm_höhe = fenster.winfo_screenheight()  # Bildschirmhöhe abrufen
    x_position = (bildschirm_breite - breite) // 2  # X-Position des Fensters berechnen
    y_position = (bildschirm_höhe - höhe) // 2  # Y-Position des Fensters berechnen
    fenster.geometry(f"{breite}x{höhe}+{x_position}+{y_position}")  # Fenstergröße und -position festlegen

# Funktion zum Einstellen des Hintergrunds für ein Fenster
def hintergrund_einstellen(fenster, farbe):
    fenster.configure(background=farbe)  # Hintergrundfarbe des Fensters festlegen

# Funktion zur Berechnung des Gesamtpreises
def preis_berechnen():
    aufenthalt_auswahl = combobox_aufenthalt.get()
    status_auswahl = combobox_status.get()
    zusatz_auswahl = combobox_zusatz.get()
    datum = calendar.get_date()
    monat = int(datum.split('.')[1])
    
    # Startzeit-Eingabe validieren
    while True:
        startzeit = entry_startzeit.get()  # Startzeit aus Eingabefeld holen
        
        # Wenn keine Startzeit eingegeben wurde
        if not startzeit:
            messagebox.showerror("Fehler", "Bitte geben Sie eine Startzeit ein.")
            return
        
        try:
            stunden, minuten = map(int, startzeit.split(':'))  # Startzeit in Stunden und Minuten aufteilen und in Integer umwandeln
            
            # Gültigkeit der Startzeit prüfen
            if not (9 <= stunden <= 20) or not (0 <= minuten <= 59):
                messagebox.showerror("Fehler", "Bitte geben Sie eine gültige Startzeit zwischen 09:00 und 20:00 ein.")
                return
        except ValueError:
            messagebox.showerror("Fehler", "Ungültiges Zeitformat. Bitte geben Sie die Startzeit im Format HH:MM ein.")
            continue  # Bei ungültigem Format zurück zur Eingabe der Startzeit
        
        break  # Gültige Startzeit wurde eingegeben, Schleife beenden


    if aufenthalt_auswahl not in STANDARDPREIS:
        messagebox.showerror("Fehler", "Bitte wählen Sie eine gültige Aufenthaltsdauer aus.")
        return

    preis1 = STANDARDPREIS[aufenthalt_auswahl]  # Standardpreis für die ausgewählte Aufenthaltsdauer abrufen

    if monat in WINTERMONATE:  # Überprüfen, ob der Monat in der Winterzeit liegt
        preis2 = preis1 * (1 - RABATT_WINTERZEIT)  # Rabatt während der Winterzeit anwenden
    else:
        preis2 = preis1  # Kein Rabatt anwenden

    if status_auswahl in RABATT:  # Überprüfen, ob die Personengruppe einen Rabatt erhält
        preis2 *= (1 - RABATT[status_auswahl])  # Rabatt für die Personengruppe anwenden

    if zusatz_auswahl in ZUSATZPREIS:  # Überprüfen, ob eine Zusatzoption gewählt wurde
        preis2 += ZUSATZPREIS[zusatz_auswahl]  # Preis für die Zusatzoption hinzufügen

    gesamt_preis.append(preis2)  # Berechneten Preis zur Liste hinzufügen
    tickets.append((preis2, aufenthalt_auswahl, status_auswahl, zusatz_auswahl, datum, startzeit))
        
    if not messagebox.askyesno("Noch ein Ticket", "Möchten Sie noch ein weiteres Ticket hinzufügen?"):
        zahlung(sum(gesamt_preis))  # Die Zahlungsfunktion wird aufgerufen und der Gesamtpreis aller Tickets übergeben.
    else:
        reset_form()  # Andernfalls wird das Formular zurückgesetzt.


# Funktion zur Anzeige der Zahlungsübersicht
def zahlung(gesamtpreis):
    zahlungsfenster = tk.Toplevel(window)
    zahlungsfenster.title("Zahlungsinformationen")
    hintergrund_einstellen(zahlungsfenster, "#add8e6")  # Hintergrundfarbe für das Zahlungsfenster setzen
    fenster_einstellen(zahlungsfenster, 800, 600)  # Fenstergröße für das Zahlungsfenster setzen

    ttk.Label(zahlungsfenster, text="Zahlungsinformationen", font=("Helvetica", 16), background='#add8e6').pack(pady=10)  # Titel Label für Zahlungsinformationen
    ttk.Label(zahlungsfenster, text=f"Gesamtbetrag: {gesamtpreis:.2f} Euro", background='#add8e6').pack()  # Label für den Gesamtbetrag

    for ticket in tickets:
        preis, aufenthalt_auswahl, status_auswahl, zusatz_auswahl, datum, startzeit = ticket
        ttk.Label(zahlungsfenster, text=f"Ticket: {preis:.2f} Euro", background='#add8e6').pack()  # Label für den Ticketpreis
        ttk.Label(zahlungsfenster, text=f"Aufenthaltsdauer: {aufenthalt_auswahl}", background='#add8e6').pack()  # Label für die Aufenthaltsdauer
        ttk.Label(zahlungsfenster, text=f"Personengruppe: {status_auswahl}", background='#add8e6').pack()  # Label für die Personengruppe
        ttk.Label(zahlungsfenster, text=f"Zusatzoption: {zusatz_auswahl}", background='#add8e6').pack()  # Label für die Zusatzoption
        ttk.Label(zahlungsfenster, text=f"Besuchsdatum: {datum}", background='#add8e6').pack()  # Label für das Besuchsdatum
        ttk.Label(zahlungsfenster, text=f"Startzeit: {startzeit}", background='#add8e6').pack()  # Label für die Startzeit
        ttk.Label(zahlungsfenster, text="--------------------------------------------", background='#add8e6').pack()  # Trennlinie

    ttk.Label(zahlungsfenster, text="Zahlungsart auswählen:", background='#add8e6').pack(pady=(20, 5))  # Label für die Auswahl der Zahlungsart

    def rechnung_anzeigen(zahlungsmethode):
        rechnungsfenster = tk.Toplevel(window)
        rechnungsfenster.title("Rechnung")
        hintergrund_einstellen(rechnungsfenster, "#add8e6")  # Hintergrundfarbe für das Rechnungsfenster setzen
        fenster_einstellen(rechnungsfenster, 800, 600)  # Fenstergröße für das Rechnungsfenster setzen

        ttk.Label(rechnungsfenster, text="Rechnung", font=("Helvetica", 16), background='#add8e6').pack(pady=10)  # Titel Label für Rechnung
        ttk.Label(rechnungsfenster, text=f"Gesamtbetrag: {gesamtpreis:.2f} Euro", background='#add8e6').pack()  # Label für den Gesamtbetrag
        ttk.Label(rechnungsfenster, text="--------------------------------------------", background='#add8e6').pack()  # Trennlinie

        for ticket in tickets:
            preis, aufenthalt_auswahl, status_auswahl, zusatz_auswahl, datum, startzeit = ticket
            ttk.Label(rechnungsfenster, text=f"Ticket: {preis:.2f} Euro", background='#add8e6').pack()  # Label für den Ticketpreis
            ttk.Label(rechnungsfenster, text=f"Aufenthaltsdauer: {aufenthalt_auswahl}", background='#add8e6').pack()  # Label für die Aufenthaltsdauer
            ttk.Label(rechnungsfenster, text=f"Personengruppe: {status_auswahl}", background='#add8e6').pack()  # Label für die Personengruppe
            ttk.Label(rechnungsfenster, text=f"Zusatzoption: {zusatz_auswahl}", background='#add8e6').pack()  # Label für die Zusatzoption
            ttk.Label(rechnungsfenster, text=f"Besuchsdatum: {datum}", background='#add8e6').pack()  # Label für das Besuchsdatum
            ttk.Label(rechnungsfenster, text=f"Startzeit: {startzeit}", background='#add8e6').pack()  # Label für die Startzeit
            ttk.Label(rechnungsfenster, text="--------------------------------------------", background='#add8e6').pack()  # Trennlinie
            
        ttk.Label(rechnungsfenster, text=f"Zahlungsart: {zahlungsmethode}", background='#add8e6').pack(pady=10)  # Label für die gewählte Zahlungsart
        ttk.Button(rechnungsfenster, text="Schließen", command=lambda: [rechnungsfenster.destroy(), zahlungsfenster.destroy(), window.destroy()]).pack(pady=10)  # Schließen Button für das Rechnungsfenster

    ttk.Button(zahlungsfenster, text="PayPal", command=lambda: rechnung_anzeigen("PayPal")).pack(pady=5)  # Button für PayPal Zahlung
    ttk.Button(zahlungsfenster, text="Kreditkarte", command=lambda: rechnung_anzeigen("Kreditkarte")).pack(pady=5)  # Button für Kreditkarte Zahlung
    ttk.Button(zahlungsfenster, text="EC-Karte", command=lambda: rechnung_anzeigen("EC-Karte")).pack(pady=5)  # Button für EC-Karte Zahlung

    # Funktion zum Zurücksetzen des Formulars
def reset_form():
    combobox_aufenthalt.set('')  # Auswahl für Aufenthaltsdauer zurücksetzen
    combobox_status.set('')  # Auswahl für Personengruppe zurücksetzen
    combobox_zusatz.set('')  # Auswahl für Zusatzoptionen zurücksetzen
    today = datetime.now().date()  # Heutiges Datum erhalten
    calendar.selection_set(today)  # Kalender auf das heutige Datum setzen
    entry_startzeit.delete(0, tk.END)  # Eingabefeld für Startzeit löschen

# Hauptfenster anpassen
fenster_einstellen(window, 800, 600)

# Stil für Widgets anpassen
style = ttk.Style()
style.configure("Custom.TFrame", background="#add8e6")  # Stil für benutzerdefiniertes Frame
style.configure("Custom.TLabel", background="#add8e6")  # Stil für benutzerdefiniertes Label

# Hintergrundbild setzen
hintergrund_einstellen(window, "#add8e6")

# Stil für Widgets anpassen
style = ttk.Style()
style.configure("Custom.TFrame", background="#add8e6")
style.configure("Custom.TLabel", background="#add8e6")

# Hintergrundbild setzen
hintergrund_einstellen(window, "#add8e6")

# Rahmen für Eingabeformular erstellen
frame = ttk.Frame(window, padding="10", style="Custom.TFrame")
frame.pack(expand=True)

# Labels und Dropdown-Menüs für das Eingabeformular
label_zusatz = ttk.Label(frame, text="Möchten Sie Zusatzoptionen?", background="#add8e6")
label_zusatz.grid(row=0, column=0, padx=5, pady=5)

combobox_zusatz = ttk.Combobox(frame)
combobox_zusatz['values'] = ZUSATZ
combobox_zusatz.grid(row=1, column=0, padx=5, pady=5)

label_aufenthalt = ttk.Label(frame, text="Wie lange planen Sie Ihren Aufenthalt bei uns?", background="#add8e6")
label_aufenthalt.grid(row=2, column=0, padx=5, pady=5)

combobox_aufenthalt = ttk.Combobox(frame)
combobox_aufenthalt['values'] = AUFENTHALT
combobox_aufenthalt.grid(row=3, column=0, padx=5, pady=5)

label_status = ttk.Label(frame, text="Zu welcher Personengruppe gehören Sie?", background="#add8e6")
label_status.grid(row=4, column=0, padx=5, pady=5)

combobox_status = ttk.Combobox(frame)
combobox_status['values'] = STATUS
combobox_status.grid(row=5, column=0, padx=5, pady=5)

label_calendar = ttk.Label(frame, text="Welchen Tag besuchen Sie uns?", background="#add8e6")
label_calendar.grid(row=6, column=0, padx=5, pady=5)

heute = datetime.now()
max_date = heute + timedelta(days=60)

calendar = Calendar(frame, selectmode='day', date_pattern='dd.mm.yyyy',
                    background='lightblue', foreground='black',
                    selectbackground='lightblue', selectforeground='white', mindate=heute, maxdate=max_date)
calendar.grid(row=7, column=0, padx=5, pady=5)

# Label und Eingabefeld für Startzeit
label_startzeit = ttk.Label(frame, text="Startzeit (HH:MM) - 09:00-20:00", style="Custom.TLabel")
label_startzeit.grid(row=8, column=0, padx=5, pady=5)

entry_startzeit = ttk.Entry(frame)
entry_startzeit.grid(row=9, column=0, padx=5, pady=5)

button = ttk.Button(frame, text="Ticket hinzufügen", command=preis_berechnen)
button.grid(row=10, column=0, padx=5, pady=5)

# Hauptfenster ausführen
window.mainloop()
